In [18]:
import os
import pandas as pd

datapath = "../data/raw"

In [77]:
df1 = pd.read_csv(os.path.join(datapath, "chembl_uniprot_mapping.txt"), sep="\t")
df2 = pd.read_csv(os.path.join(datapath, "mtuberculosis_ChEMBL_data.csv"))

In [78]:
df2.columns

Index(['activity_id', 'assay_id', 'assay_chembl_id', 'assay_type',
       'assay_confidence_score', 'assay_organism', 'doc_chembl_id', 'tid',
       'target_type', 'target_organism', 'target_chembl_id', 'target_tax_id',
       'compound_chembl_id', 'canonical_smiles', 'MW', 'pchembl',
       'activity_comment', 'standard_text', 'value', 'unit', 'activity_type',
       'relation', 'pchembl_calculated'],
      dtype='object')

In [79]:
# Is binding for proteins and functional for organism?
print("Functional",  df2[df2["assay_type"]=="F"]["target_type"].value_counts())
print("Binding", df2[df2["assay_type"]=="B"]["target_type"].value_counts())

Functional target_type
SINGLE PROTEIN     217253
ORGANISM           211230
UNCHECKED          200094
PROTEIN COMPLEX     69290
NON-MOLECULAR         273
SUBCELLULAR             8
Name: count, dtype: int64
Binding target_type
SINGLE PROTEIN     8932
UNCHECKED          5999
PROTEIN COMPLEX     589
ORGANISM            328
SUBCELLULAR          30
PROTEIN FAMILY        1
Name: count, dtype: int64


In [81]:
df_ = df2[df2["target_type"]=="PROTEIN COMPLEX"]
df_["target_chembl_id"].nunique()

3

In [29]:
# Unchecked counts
df_ = df2[df2["target_type"]=="UNCHECKED"]
df_ = df_["assay_chembl_id"].value_counts()
df_ = df_.reset_index()
df_.columns = ["assay_chembl_id", "counts"]
df_.to_csv("../data/processed/chembl/chembl_unchecked.csv", index=False)

In [41]:
#manual curation 
# DO NOT DELETE
df = pd.read_csv("../data/processed/chembl/chembl_unchecked_manual.csv")
df = df[~df["target_manual"].isna()]
df = df[["assay_chembl_id", "target_chembl_id"]].drop_duplicates(keep="first")

In [42]:
len(set(df["target_chembl_id"].tolist()))

15

In [43]:
#mapping from df to the mtuberculosis_ChEMBL_data
df2 = pd.read_csv(os.path.join(datapath, "mtuberculosis_ChEMBL_data.csv"))
pre = (set(df2["target_chembl_id"].tolist()))
lookup = df.set_index("assay_chembl_id")["target_chembl_id"]
mapped = df2["assay_chembl_id"].map(lookup)
df2["target_chembl_id"] = mapped.fillna(df2["target_chembl_id"])
post = (set(df2["target_chembl_id"].tolist()))

In [44]:
post-pre

{'UNKNOWN'}

In [45]:
df2.to_csv("../data/processed/chembl/mtuberculosis_chembl_edited.csv", index=False)

# Generate merging file target_chembl_id to uniprot_ac

In [94]:
df1 = pd.read_csv(os.path.join(datapath, "chembl_uniprot_mapping.txt"), sep="\t")
df2 = pd.read_csv("../data/raw/mtuberculosis_ChEMBL_data.csv")
targets = list(set(df2["target_chembl_id"].tolist()))
len(targets)

146

In [95]:
df_ = (
    df1[df1["chembl_id"].isin(targets)]
      .dropna(subset=["uniprot_ac"])                   
      .rename(columns={"chembl_id": "target_chembl_id"})
)
df_["target_chembl_id"].value_counts()

target_chembl_id
CHEMBL2364166    8
CHEMBL4662931    2
CHEMBL3430898    2
CHEMBL3883301    2
CHEMBL2361       1
                ..
CHEMBL6086       1
CHEMBL6070       1
CHEMBL6069       1
CHEMBL6067       1
CHEMBL3988585    1
Name: count, Length: 136, dtype: int64

In [96]:
#add manual targets
df = pd.read_csv("../data/processed/chembl/chembl_unchecked_manual.csv")
df = df[~df["uniprot_ac"].isna()]
df = df[df["uniprot_ac"]!="UNKNOWN"]
df=df[["target_chembl_id","uniprot_ac"]]

In [97]:
df_uniprots = list(set(df["uniprot_ac"].tolist()))
df_uniprots2 = list(set(df_["uniprot_ac"].tolist()))

In [98]:
print(len(df_uniprots), len(df_uniprots2), len(set(df_uniprots+df_uniprots2)))

32 144 154


In [99]:
uniprot_ac_in_chembl = pd.DataFrame({"uniprot_ac":list(set(df_uniprots+df_uniprots2))})
uniprot_ac_in_chembl.to_csv(("../data/processed/chembl/uniprot_ac_in_chembl.csv"), index=False)

# Molecule - ChEMBL targets list

In [46]:
import pandas as pd

df = pd.read_csv("../data/processed/chembl/mtuberculosis_chembl_edited.csv")
print(df["compound_chembl_id"].nunique())
print(df["canonical_smiles"].nunique())

132378
132378


In [47]:

targets = list(set(df["target_chembl_id"].tolist()))
len(targets)

147

In [67]:
df_sub = df[["canonical_smiles", "compound_chembl_id", "target_chembl_id"]].copy()

counts = (
    df_sub
    .groupby(["canonical_smiles", "compound_chembl_id", "target_chembl_id"])
    .size()
    .reset_index(name="count")
)

pivot = counts.pivot_table(
    index=["canonical_smiles", "compound_chembl_id"],
    columns="target_chembl_id",
    values="count",
    aggfunc="sum",
    fill_value=0,
)

pivot.columns = pivot.columns.astype(str)

pivot = pivot.reset_index()
pivot = pivot.rename_axis(None, axis="index")

In [55]:
pivot.to_csv("../data/processed/chembl/mols2target.csv",index=False)

### map chembl id to uniprot ac

In [49]:
import pandas as pd

df1 = pd.read_csv("../data/raw/chembl_uniprot_mapping.txt", sep="\t")
df2 = pd.read_csv("../data/processed/chembl/mtuberculosis_chembl_edited.csv")
targets = list(set(df2["target_chembl_id"].tolist()))
print(len(targets))
df1=df1[df1["chembl_id"].isin(targets)]
df1 = df1[["chembl_id","uniprot_ac"]]
set(targets)-set(df1["chembl_id"])

147


{'CHEMBL2111188',
 'CHEMBL2364031',
 'CHEMBL360',
 'CHEMBL3879801',
 'CHEMBL612545',
 'CHEMBL612558',
 'CHEMBL612960',
 'CHEMBL613086',
 'CHEMBL614987',
 'CHEMBL615052',
 'UNKNOWN'}

In [50]:
other = {'CHEMBL2111188':"Mycobacterium tuberculosis H37Rv",
 'CHEMBL2364031':"Cell membrane",
 'CHEMBL360':"Mycobacterium tuberculosis",
 'CHEMBL3879801':"NON-PROTEIN TARGET",
 'CHEMBL612545':"UNCHECKED",
 'CHEMBL612558':"ADMET",
 'CHEMBL612960':"Mycobacterium tuberculosis variant microti",
 'CHEMBL613086':"Mycobacterium tuberculosis variant bovis",
 'CHEMBL614987':"Mycobacterium marinum",
 'CHEMBL615052':"Mycobacterium tuberculosis variant bovis BCG",
 'UNKNOWN':"UNCHECKED"}

In [51]:
df_other = pd.DataFrame(
    [(k, v) for k, v in other.items()],
    columns=["chembl_id", "uniprot_ac"]
)

In [52]:
df1 = pd.concat([df1,df_other],ignore_index=True)
df1.to_csv("../data/processed/chembl/chembl2uniprot.csv", index=False)

### Chembl IDs to Uniprot AC

Use whole name of organism if no uniprot exists

In [53]:
import pandas as pd
df = pd.read_csv("../data/processed/chembl/mols2target.csv")
df_ = pd.read_csv("../data/processed/chembl/chembl2uniprot.csv")
mapping = (
    df_
    .dropna(subset=["uniprot_ac"])
    .drop_duplicates(subset=["chembl_id", "uniprot_ac"])
    .groupby("chembl_id")["uniprot_ac"]
    .apply(list)
    .to_dict()
)
print(df.shape)

(132378, 149)


In [54]:
import pandas as pd

def merge_targets_to_uniprot_sum(pivot: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Given:
      - pivot: DataFrame with columns = CHEMBL target IDs (plus possibly other cols)
      - mapping: dict {chembl_id: [uniprot_ac1, uniprot_ac2, ...]}

    Returns:
      - new DataFrame where:
          * columns are UniProt ACs (and unmapped CHEMBL IDs)
          * if multiple CHEMBL IDs map to the same UniProt, they are summed
          * if a CHEMBL ID maps to multiple UniProts, its values are added to each
          * CHEMBL IDs with no mapping are kept as separate columns
    """
    # This dict will accumulate Series per output column name (UniProt or CHEMBL)
    new_cols = {}

    for col in pivot.columns:
        # CHEMBL target not in mapping: keep as-is
        if col not in mapping:
            print("Not found", col)
            if col in new_cols:
                # if somehow duplicated, add
                new_cols[col] = new_cols[col] + pivot[col]
            else:
                new_cols[col] = pivot[col]
            continue

        # For mapped CHEMBL IDs, add their values into one or more UniProt columns
        uniprots = mapping[col]
        if not uniprots:
            print("No uniprot entry", col)
            # No UniProt entries → treat as unmapped CHEMBL
            if col in new_cols:
                new_cols[col] = new_cols[col] + pivot[col]
            else:
                new_cols[col] = pivot[col]
        else:
            for up in uniprots:
                if up in new_cols:
                    print("Duplicated?", col)
                    new_cols[up] = new_cols[up] + pivot[col]
                else:
                    new_cols[up] = pivot[col]

    # Build the final DataFrame (aligned on the same index as pivot)
    pivot_u = pd.DataFrame(new_cols, index=pivot.index)

    return pivot_u

In [55]:
df_uniprot = merge_targets_to_uniprot_sum(df, mapping)
df_uniprot = df_uniprot.reset_index(drop=True)

Not found canonical_smiles
Not found compound_chembl_id
Duplicated? CHEMBL3883301
Duplicated? CHEMBL4165
Duplicated? UNKNOWN


In [56]:
len(set(df_uniprot.columns))

156

In [57]:
for x in df_uniprot.columns:
    if "P9WG47" in x:
        print(x)

P9WG47


In [58]:
df_uniprot["P9WG47"].sum()

688

In [59]:
df_uniprot["UNCHECKED"].sum()

5020

In [ ]:
df_uniprot[df_uniprot["CHEMBL360"]]

,canonical_smiles,compound_chembl_id,O53585,P9WHH3,P9WHE9,P9WP01,P9WNP5,P9WQA3,P9WMN3,P9WKK7,...,P9WIL5,P9WI73,P9WMN1,UNCHECKED,ADMET,Mycobacterium tuberculosis variant microti,Mycobacterium tuberculosis variant bovis,Mycobacterium marinum,Mycobacterium tuberculosis variant bovis BCG,P9WHV7
0,Br.Br.CC[C@@H](CO)NCCN[C@@H](CC)CO,CHEMBL1788373,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Br.CC(/C=C/c1cnc([N+](=O)[O-])n1C)=N\NC1=NCCN1,CHEMBL3247264,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Br.CC(C)(C)c1ccccc1OCCCON1C(=N)N=C(N)NC1(C)C,CHEMBL588011,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Br.CC1(C)N=C(N)N=C(N)N1OCCc1ccc2ccccc2c1,CHEMBL528829,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Br.CC1(C)NC(N)=NC(=N)N1OCCCOc1ccc(Cl)cc1Cl,CHEMBL2098379,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132373,c1sc(C2CCCCC2)nc1CN1CCn2c(nnc2C2CC2)C1,CHEMBL3489159,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132374,c1snnc1CN1CCC(OCC2CCOC2)CC1,CHEMBL4563654,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132375,c1snnc1CN1CCC[C@H]2OCC[C@H]21,CHEMBL4544987,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132376,c1snnc1CN1CCOCC2(CCOCC2)C1,CHEMBL4592194,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df_uniprot.to_csv("../data/processed/chembl/mols2uniprot.csv", index=False)

In [93]:
from tqdm import tqdm
from rdkit import Chem
import numpy as np
from standardiser import standardise

df_uniprot = pd.read_csv("../data/processed/chembl/mols2uniprot.csv")

def standardise_smiles(df, smi_col):
    std_smiles = []
    for smiles in tqdm(list(df[smi_col])):
        try:
            mol = Chem.MolFromSmiles(smiles)
            mol = standardise.run(mol)
            std_smiles.append(Chem.MolToSmiles(mol))
        except:
            std_smiles.append(None)
    df["std_smiles"] = std_smiles
    df = df[df["std_smiles"].notnull()]
    return df

df = standardise_smiles(df_uniprot, "canonical_smiles")
df = df.drop(columns=["canonical_smiles"])
print(df.shape)

 19%|█▉        | 24925/132378 [00:33<02:31, 711.28it/s][13:08:21] Can't kekulize mol.  Unkekulized atoms: 3 7
[13:08:21] Can't kekulize mol.  Unkekulized atoms: 3 7
 21%|██        | 27760/132378 [00:36<02:10, 800.47it/s][13:08:25] Can't kekulize mol.  Unkekulized atoms: 3 7
[13:08:25] Can't kekulize mol.  Unkekulized atoms: 3 7
 26%|██▋       | 34846/132378 [00:46<02:36, 624.90it/s][13:08:34] Can't kekulize mol.  Unkekulized atoms: 2 5
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 2 5
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 2 5
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 3 9
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 3 9
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 3 9
[13:08:34] Can't kekulize mol.  Unkekulized atoms: 3 9
 54%|█████▎    | 70932/132378 [01:35<01:17, 796.92it/s][13:09:24] Can't kekulize mol.  Unkekulized atoms: 3 12
[13:09:24] Can't kekulize mol.  Unkekulized atoms: 3 9
[13:09:24] Can't kekulize mol.  Unkekulized atoms: 3 9
 83%|████

(132094, 156)


In [94]:
df["std_smiles"].nunique()

132020

In [95]:
df["std_smiles"].value_counts()

std_smiles
CC[C@@H](CO)NCCN[C@@H](CC)CO                                           3
COc1cc(NC(C)CCCN)c2ncccc2c1                                            3
Nc1ncnc2c1ncn2[C@@H]1O[C@H](COS(=O)(=O)NCc2ccccc2O)[C@@H](O)[C@H]1O    2
O=C(CCCc1ccccc1)N(O)CCCP(=O)(O)O                                       2
COc1ccc2c(c[n+](C)c3c4cc5c(cc4ccc23)OCO5)c1OC                          2
                                                                      ..
CN(Cc1ccncn1)C(=O)CCc1[nH]nc2c1CCCC2                                   1
CN(Cc1ccncn1)C(=O)CCC(=O)NC1CCCC1                                      1
CN(Cc1ccncn1)C(=O)C1(Oc2ccc(Cl)cc2)CCNCC1                              1
CN(Cc1ccncc1)c1cncc(Br)c1                                              1
c1snnc1CNCC1COC2(CCCCC2)O1                                             1
Name: count, Length: 132020, dtype: int64

In [96]:
df[df["std_smiles"]=="CC[C@@H](CO)NCCN[C@@H](CC)CO"]

,compound_chembl_id,O53585,P9WHH3,P9WHE9,P9WP01,P9WNP5,P9WQA3,P9WMN3,P9WKK7,P9WPP7,...,P9WI73,P9WMN1,UNCHECKED,ADMET,Mycobacterium tuberculosis variant microti,Mycobacterium tuberculosis variant bovis,Mycobacterium marinum,Mycobacterium tuberculosis variant bovis BCG,P9WHV7,std_smiles
0,CHEMBL1788373,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CC[C@@H](CO)NCCN[C@@H](CC)CO
34339,CHEMBL44884,0,0,0,0,0,0,0,0,0,...,0,0,7,0,0,9,0,2,0,CC[C@@H](CO)NCCN[C@@H](CC)CO
34340,CHEMBL3140361,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CC[C@@H](CO)NCCN[C@@H](CC)CO


In [97]:
value_cols = df.columns.difference(["std_smiles", "compound_chembl_id"])

compound_sums = (
    df.groupby("compound_chembl_id")[value_cols]
      .sum()
      .sum(axis=1)
)

def pick_best_compound(series):
    return compound_sums.loc[series].idxmax()

In [99]:
compound_sums.max()

2683

In [100]:
collapsed = df_uniprot.groupby("std_smiles").agg(
    {**{col: "sum" for col in value_cols},          # sum all numeric columns
     "compound_chembl_id": pick_best_compound}      # choose best compound ID
).reset_index()

In [101]:
collapsed["sum"] = collapsed[value_cols].sum(axis=1)

In [103]:
print("Before:", len(df))
print("After :", len(collapsed))

Before: 132094
After : 132020


In [104]:
collapsed.to_csv("../data/processed/chembl/mols2uniprot_st.csv", index=False)